# Import bibliotek

In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models, activations
from tensorflow.python.keras.callbacks import ModelCheckpoint
import torch

In [2]:
gpus = tf.config.list_physical_devices()
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

Name: /physical_device:CPU:0   Type: CPU
Name: /physical_device:GPU:0   Type: GPU


# Zmienne

In [3]:
SPECTROGRAM = False
classes = ["Anger", "Happy", "Neutral", "Sad", "Fear", "Disgust"]

In [4]:
if SPECTROGRAM:
    path_train = "../data/spectrograms/divided/Ravdess/train"
    path_test = "../data/spectrograms/divided/Ravdess/test"
    log_directory = "../tests/logs/TensorBoard/spec_ravdess"
    filepath = "../tests/logs/SavedModels/spec_ravdess.h5"

else:
    path_train = "../data/melspectrograms/divided/Ravdess/train"
    path_test = "../data/melspectrograms/divided/Ravdess/test"
    log_directory = "../tests/logs/TensorBoard/melspec_ravdess"
    filepath = "../tests/logs/SavedModels/melspec_ravdess.h5"

In [5]:
def get_train_data(train_data_path):
    """
    Loads train data from all datasets

    Returns:
        data_train - training samples
        data_val - validation samples
        target_train - training targets
        target_val - validation targets
        
    """
    class_number = 0
    targets = []
    img = []

    for current_folder in classes:
        print(f"Emotion: {current_folder}")
        emotion_folder = f"{train_data_path}/{current_folder}"
        for i, file_name in enumerate(os.listdir(emotion_folder)):
            file_path = f"{emotion_folder}/{file_name}"
            targets.append(class_number)
            img.append(np.array(Image.open(file_path).convert('RGB'))/255)
            print(f"Loaded {i+1}/{len(os.listdir(emotion_folder))} files from train {current_folder}", end="\r")
        print("\n")
        class_number += 1

    
    targets_array = np.asarray(targets)
    print("Set np.asarray(targets)")
    targets = []
    img_array = np.asarray(img)
    print("Set np.asarray(img)")
    img = []

    targets_array = tf.keras.utils.to_categorical(targets_array)
    print("Set tf.keras.utils.to_categorical(targets_array)")
    
    data_train, data_val, target_train, target_val = train_test_split(img_array, targets_array, test_size=0.25, random_state=0)
    print("Splitted to train/val datasets")
    
    print("shapes")
    print(data_train.shape)
    print(target_train.shape)
    print(data_val.shape)
    print(target_val.shape)

    return data_train, data_val, target_train, target_val

In [6]:
def get_test_data(test_data_path):
    """
    Loads test data from all datasets

    Returns:
        data_test - test samples
        target_test_to_categorical - test targets
        
    """
    class_number = 0
    targets = []
    img = []

    for current_folder in classes:
        print(f"Emotion: {current_folder}")
        emotion_folder = f"{test_data_path}/{current_folder}"
        for i, file_name in enumerate(os.listdir(emotion_folder)):
            file_path = f"{emotion_folder}/{file_name}"
            targets.append(class_number)
            img.append(np.array(Image.open(file_path).convert('RGB'))/255)
            print(f"Loaded {i+1}/{len(os.listdir(emotion_folder))} files from test {current_folder}", end="\r")
        print("\n")
        class_number += 1

    target_test = np.asarray(targets)
    print("Set np.asarray(targets)")
    targets = []
    data_test = np.asarray(img)
    print("Set np.asarray(img)")
    img = []
    
    target_test_to_categorical = tf.keras.utils.to_categorical(target_test)
    print("Set tf.keras.utils.to_categorical(target_test)")

    print(data_test.shape)
    print(target_test_to_categorical.shape)

    return data_test, target_test_to_categorical

In [7]:
# MODEL SHEDULER
def scheduler(epoch, lr):
    if epoch < 15:
        eta = lr
    else:
        eta = lr - 0.00001
    if lr < 0.00005:
        eta = 0.00005

    return eta

In [8]:
data_train, data_val, target_train, target_val = get_train_data(path_train)

Emotion: Anger
Loaded 153/153 files from train Anger

Emotion: Happy
Loaded 153/153 files from train Happy

Emotion: Neutral
Loaded 77/77 files from train Neutral

Emotion: Sad
Loaded 154/154 files from train Sad

Emotion: Fear
Loaded 153/153 files from train Fear

Emotion: Disgust
Loaded 154/154 files from train Disgust

Set np.asarray(targets)
Set np.asarray(img)
Set tf.keras.utils.to_categorical(targets_array)
Splitted to train/val datasets
shapes
(633, 389, 515, 3)
(633, 6)
(211, 389, 515, 3)
(211, 6)


In [9]:
# MODEL SHEDULER
scheduler_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

# TENSORBOARD
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_directory)

# SAVE MODEL 
checkpoint = ModelCheckpoint(filepath,
                             monitor='val_loss', 
                             verbose=1,
                             save_best_only=True,
                             mode='min')


model = models.Sequential()
model.add(layers.Conv2D(75, (5, 5), activation='relu', input_shape=(389, 515, 3), padding='same'))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Conv2D(135, (5, 5), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Dropout(0.15))
model.add(layers.Conv2D(75, (5, 5), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Dropout(0.25))
model.add(layers.Flatten()) 
model.add(layers.Dense(45, activation='relu'))
model.add(layers.Dropout(0.2))
# model.add(layers.Dense(40, activation='relu'))
# model.add(layers.Dropout(0.3))
model.add(layers.Dense(6, activation='softmax'))


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 389, 515, 75)      5700      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 129, 171, 75)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 129, 171, 135)     253260    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 43, 57, 135)      0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 43, 57, 135)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 43, 57, 75)        2

In [10]:
# MODEL COMPILE 
model.compile(optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])

In [11]:
# MODEL FIT 
history = model.fit(data_train,
                    target_train,
                    epochs=20,
                    shuffle=True,
                    validation_data=(data_val, target_val),
                    callbacks=[tensorboard_callback, checkpoint])

Epoch 1/20


2024-10-29 14:08:02.983386: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


20/20 [==============================] - ETA: 0s - loss: 1.8525 - accuracy: 0.1785
Epoch 00001: val_loss improved from inf to 1.75254, saving model to ../tests/logs/SavedModels/melspec_ravdess.h5
20/20 [==============================] - 78s 4s/step - loss: 1.8525 - accuracy: 0.1785 - val_loss: 1.7525 - val_accuracy: 0.1611
Epoch 2/20
20/20 [==============================] - ETA: 0s - loss: 1.7538 - accuracy: 0.2148
Epoch 00002: val_loss improved from 1.75254 to 1.72478, saving model to ../tests/logs/SavedModels/melspec_ravdess.h5
20/20 [==============================] - 75s 4s/step - loss: 1.7538 - accuracy: 0.2148 - val_loss: 1.7248 - val_accuracy: 0.3033
Epoch 3/20
20/20 [==============================] - ETA: 0s - loss: 1.7815 - accuracy: 0.2638
Epoch 00003: val_loss improved from 1.72478 to 1.62138, saving model to ../tests/logs/SavedModels/melspec_ravdess.h5
20/20 [==============================] - 67s 3s/step - loss: 1.7815 - accuracy: 0.2638 - val_loss: 1.6214 - val_accuracy: 0.

Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Internal Error (0000000e:Internal Error)
	<AGXG13XFamilyCommandBuffer: 0x34c7ad850>
    label = <none> 
    device = <AGXG13XDevice: 0x13aee9000>
        name = Apple M1 Pro 
    commandQueue = <AGXG13XFamilyCommandQueue: 0x104248200>
        label = <none> 
        device = <AGXG13XDevice: 0x13aee9000>
            name = Apple M1 Pro 
    retainedReferences = 1


20/20 [==============================] - ETA: 0s - loss: 1.4345 - accuracy: 0.4408
Epoch 00008: val_loss did not improve from 1.35017
20/20 [==============================] - 73s 4s/step - loss: 1.4345 - accuracy: 0.4408 - val_loss: 1.3584 - val_accuracy: 0.5166
Epoch 9/20
20/20 [==============================] - ETA: 0s - loss: 1.3249 - accuracy: 0.4771
Epoch 00009: val_loss improved from 1.35017 to 1.25794, saving model to ../tests/logs/SavedModels/melspec_ravdess.h5
20/20 [==============================] - 46s 2s/step - loss: 1.3249 - accuracy: 0.4771 - val_loss: 1.2579 - val_accuracy: 0.5024
Epoch 10/20
20/20 [==============================] - ETA: 0s - loss: 1.3168 - accuracy: 0.4803
Epoch 00010: val_loss improved from 1.25794 to 1.18220, saving model to ../tests/logs/SavedModels/melspec_ravdess.h5
20/20 [==============================] - 37s 2s/step - loss: 1.3168 - accuracy: 0.4803 - val_loss: 1.1822 - val_accuracy: 0.5403
Epoch 11/20
20/20 [==============================] - ETA

In [12]:
data_train = []
data_val = []
target_train = []
target_val = []

targets = []
img = []

data_test, target_test_to_categorical = get_test_data(path_test)

Emotion: Anger
Loaded 39/39 files from test Anger

Emotion: Happy
Loaded 39/39 files from test Happy

Emotion: Neutral
Loaded 19/19 files from test Neutral

Emotion: Sad
Loaded 38/38 files from test Sad

Emotion: Fear
Loaded 39/39 files from test Fear

Emotion: Disgust
Loaded 38/38 files from test Disgust

Set np.asarray(targets)
Set np.asarray(img)
Set tf.keras.utils.to_categorical(target_test)
(212, 389, 515, 3)
(212, 6)


In [13]:
model.load_weights(filepath=filepath)
    
results = model.evaluate(data_test, target_test_to_categorical, batch_size=1)
print("test loss, test acc:", results)

212/212 [==============================] - 5s 22ms/step - loss: 1.1894 - accuracy: 0.5708
test loss, test acc: [1.1894479990005493, 0.5707547068595886]
